<a href="https://colab.research.google.com/github/amit-chaubey/recommendation-system/blob/main/Movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/datasets/top10K-TMDB-movies.csv')

In [ ]:
df.head(3)

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280


In [ ]:
df.tail(3)

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
9997,13995,Captain America,"Action,Science Fiction,War",en,"During World War II, a brave, patriotic Americ...",18.333,1990-12-14,4.6,332
9998,2312,In the Name of the King: A Dungeon Siege Tale,"Adventure,Fantasy,Action,Drama",en,A man named Farmer sets out to rescue his kidn...,15.159,2007-11-29,4.7,668
9999,455957,Domino,"Thriller,Action,Crime",en,Seeking justice for his partner’s murder by an...,16.482,2019-05-31,4.6,221


In [ ]:
df.describe()

,id,popularity,vote_average,vote_count
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,161243.505000,34.697267,6.621150,1547.309400
std,211422.046043,211.684175,0.766231,2648.295789
min,5.000000,0.600000,4.600000,200.000000
25%,10127.750000,9.154750,6.100000,315.000000
50%,30002.500000,13.637500,6.600000,583.500000
75%,310133.500000,25.651250,7.200000,1460.000000
max,934761.000000,10436.917000,8.700000,31917.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10000 non-null  int64  
 1   title              10000 non-null  object 
 2   genre              9997 non-null   object 
 3   original_language  10000 non-null  object 
 4   overview           9987 non-null   object 
 5   popularity         10000 non-null  float64
 6   release_date       10000 non-null  object 
 7   vote_average       10000 non-null  float64
 8   vote_count         10000 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 703.2+ KB


In [ ]:
df.isnull().sum()

id                    0
title                 0
genre                 3
original_language     0
overview             13
popularity            0
release_date          0
vote_average          0
vote_count            0
dtype: int64

In [ ]:
df.columns

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')

In [ ]:
df_movie = df[['id','title', 'overview', 'genre']]

In [ ]:
df_movie.columns

Index(['id', 'title', 'overview', 'genre'], dtype='object')

In [ ]:
df_movie.head(2)

,id,title,overview,genre
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"Drama,Crime"
1,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","Comedy,Drama,Romance"


In [ ]:
df_movie.isnull().sum()

id           0
title        0
overview    13
genre        3
dtype: int64

In [ ]:
# for tags

df_movie['tags'] = df_movie['overview'] + df_movie['genre']

<ipython-input-13-cd2c94ed65b4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movie['tags'] = df_movie['overview'] + df_movie['genre']


In [ ]:
df_movie.head(2)

,id,title,overview,genre,tags
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"Drama,Crime",Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","Comedy,Drama,Romance","Raj is a rich, carefree, happy-go-lucky second..."


In [ ]:
df_new = df_movie.drop(columns= ['overview', 'genre'])
df_new

,id,title,tags
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o..."
3,424,Schindler's List,The true story of how businessman Oskar Schind...
4,240,The Godfather: Part II,In the continuing saga of the Corleone crime f...
...,...,...,...
9995,10196,The Last Airbender,"The story follows the adventures of Aang, a yo..."
9996,331446,Sharknado 3: Oh Hell No!,The sharks take bite out of the East Coast whe...
9997,13995,Captain America,"During World War II, a brave, patriotic Americ..."
9998,2312,In the Name of the King: A Dungeon Siege Tale,A man named Farmer sets out to rescue his kidn...


In [ ]:
count_vec = CountVectorizer(max_features= 20000, stop_words = 'english' )

In [ ]:
vector = count_vec.fit_transform(df_new['tags'].values.astype('U')).toarray()

In [ ]:
vector.shape

(10000, 20000)

In [ ]:
# calculate cosine similarity
tag_similarity = cosine_similarity(vector)

In [ ]:
tag_similarity

array([[1.        , 0.04652421, 0.1176552 , ..., 0.06546537, 0.10101525,
        0.05832118],
       [0.04652421, 1.        , 0.06897007, ..., 0.        , 0.03289758,
        0.        ],
       [0.1176552 , 0.06897007, 1.        , ..., 0.02156655, 0.06655583,
        0.08645856],
       ...,
       [0.06546537, 0.        , 0.02156655, ..., 1.        , 0.03086067,
        0.02672612],
       [0.10101525, 0.03289758, 0.06655583, ..., 0.03086067, 1.        ,
        0.0412393 ],
       [0.05832118, 0.        , 0.08645856, ..., 0.02672612, 0.0412393 ,
        1.        ]])

In [ ]:
distance = sorted(list(enumerate(tag_similarity[2])), reverse= True, key = lambda vector:vector[1])

In [ ]:
for dist in distance[0:3]:
  print(dist)
  print(df_new.iloc[dist[0]].title)

(2, 1.0000000000000007)
The Godfather
(4, 0.48976229911514363)
The Godfather: Part II
(7419, 0.3521803625302496)
Blood Ties


In [ ]:
def recommendation(df):
  ind = df_new[df_new['title']==df].index[0]
  distance = sorted(list(enumerate(tag_similarity[ind])), reverse= True, key = lambda vector:vector[1])
  for dist in distance[0:5]:
    print(dist)
    print(df_new.iloc[dist[0]].title)

recommendation("The Shawshank Redemption")

(0, 1.0000000000000004)
The Shawshank Redemption
(2991, 0.24743582965269678)
Brubaker
(3649, 0.2439750182371333)
The Woodsman
(9520, 0.22047927592204922)
Gotti
(698, 0.21606790785072144)
Cool Hand Luke


In [ ]:
pickle.dump(df_new , open('movies.pkl', 'wb'))
pickle.dump(tag_similarity, open('tag_similarity.pkl', 'wb'))

In [ ]:
pickle.load(open('tag_similarity.pkl', 'rb'))

array([[1.        , 0.04652421, 0.1176552 , ..., 0.06546537, 0.10101525,
        0.05832118],
       [0.04652421, 1.        , 0.06897007, ..., 0.        , 0.03289758,
        0.        ],
       [0.1176552 , 0.06897007, 1.        , ..., 0.02156655, 0.06655583,
        0.08645856],
       ...,
       [0.06546537, 0.        , 0.02156655, ..., 1.        , 0.03086067,
        0.02672612],
       [0.10101525, 0.03289758, 0.06655583, ..., 0.03086067, 1.        ,
        0.0412393 ],
       [0.05832118, 0.        , 0.08645856, ..., 0.02672612, 0.0412393 ,
        1.        ]])

In [ ]:
pickle.load(open('movies.pkl', 'rb')).head(3)

,id,title,tags
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o..."


In [ ]:
pickle.dump(df_new.to_dict(),open('movies_dic.pkl', 'wb'))